In [1]:
import pickle
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import itertools

# Get the path of the current script (or current working directory)
parent_dir = os.path.dirname(os.getcwd())
# Append the parent directory to sys.path
sys.path.append(parent_dir)

import src
from src.PrePipelineSteps import Pycromanager2NativeDataType, Make_Output_Dir_JF, Make_Analysis_Dir_JF, \
                                    ConsolidateImageShapes, TrimZSlices, AutomaticSpotDetection_JF

from src.PipelineSteps import CellSegmentationStepClass_JF, BIGFISH_SpotDetection, SimpleCellposeSegmentaion, ParamOptimizer_BIGFISH_SpotDetection, \
                                TrackPy_SpotDetection

from src.PostPipelineSteps import BuildPDFReport, SaveSpotDetectionResults, SaveMasksToAnalysis, SendAnalysisToNAS, \
                                    DeleteTempFiles, TrackPyAnlaysis

                            

from src import Experiment, PipelineSettings, ScopeClass, PipelineDataClass, Pipeline

from pycromanager import Dataset

def save_dict_to_file(dic, location):
    f = open(location,'w')
    f.write(str(dic))
    f.close()

In [2]:
ds = Dataset(r'\\munsky-nas.engr.colostate.edu\share\RNA_diffusion_data\03052024_H128_IntronDiffusion\H128FastVids_5ms_0ND_green_40mw_5')

Dataset opened                 


In [3]:
kwargs = {'nucChannel': [0], 'FISHChannel': [0],
          'user_select_number_of_images_to_run': 5, 
          'voxel_size_yx': 130,
            'voxel_size_z': 200,
          'spot_yx': 450, 
          'spot_z': 750, 
          # 'bigfish_alpha': 0.5, 
          'bigfish_beta': 1, 
          'bigfish_gamma': 5, 
          'use_log_hook': False,
          'timestep_s': (5+20) * 1e-3, # convert ms to s
          'analysis_location': None,
          'trackpy_minmass': 300,
          'sub_pixel_fitting': False,
          'trackpy_max_lagtime': 200
          }

In [4]:
compiler = src.SingleStepCompiler(ds, kwargs)

In [5]:
compiler.run_sliders([('bigfish_alpha', 'slider', (0, 1))], BIGFISH_SpotDetection)

========================== Running cell 0 ==========================
dict_keys(['nucChannel', 'FISHChannel', 'user_select_number_of_images_to_run', 'voxel_size_yx', 'voxel_size_z', 'spot_yx', 'spot_z', 'bigfish_alpha', 'bigfish_beta', 'bigfish_gamma', 'use_log_hook', 'timestep_s', 'analysis_location', 'trackpy_minmass', 'sub_pixel_fitting', 'trackpy_max_lagtime', 'list_images', 'map_id_imgprops', 'verbose', 'display_plots', 'id', 'image'])
id: 0
list_images: [array([[[[1731.],
         [1854.],
         [1720.],
         ...,
         [2903.],
         [2111.],
         [2501.]],

        [[1852.],
         [1817.],
         [2144.],
         ...,
         [2828.],
         [2805.],
         [2668.]],

        [[1717.],
         [1776.],
         [1503.],
         ...,
         [2781.],
         [2457.],
         [2974.]],

        ...,

        [[4891.],
         [4603.],
         [4425.],
         ...,
         [3526.],
         [3198.],
         [3683.]],

        [[4835.],
        

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Jack\miniconda3\envs\ImgProcessing_env\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\Jack\Documents\GitHub\FISH_Processing\src\SingleStepCompiler.py", line 29, in run_processing
    runner.sudo_run_step(function)
  File "c:\Users\Jack\Documents\GitHub\FISH_Processing\src\SingleStepCompiler.py", line 127, in sudo_run_step
    output = function.main(**kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jack\Documents\GitHub\FISH_Processing\src\PipelineSteps\PipelineSteps.py", line 593, in main
    spots_px, dense_regions, reference_spot, clusters, spots_subpx = self.bigfish_spotdetection(
                                                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jack\Documents\GitHub\FISH_Processing\src\PipelineSteps\PipelineSteps.py", line 671, in bigfish_spotdetection
    spots_post

In [6]:
output = compiler.sudo_run_step(BIGFISH_SpotDetection)
compiler.kwargs = {**compiler.kwargs, **output.__dict__}
output = compiler.sudo_run_step(TrackPyAnlaysis)

========================== Running cell 0 ==========================
dict_keys(['nucChannel', 'FISHChannel', 'user_select_number_of_images_to_run', 'voxel_size_yx', 'voxel_size_z', 'spot_yx', 'spot_z', 'bigfish_alpha', 'bigfish_beta', 'bigfish_gamma', 'use_log_hook', 'timestep_s', 'analysis_location', 'trackpy_minmass', 'sub_pixel_fitting', 'trackpy_max_lagtime', 'list_images', 'map_id_imgprops', 'verbose', 'display_plots', 'id', 'image'])
id: 0
list_images: [array([[[[1731.],
         [1854.],
         [1720.],
         ...,
         [2903.],
         [2111.],
         [2501.]],

        [[1852.],
         [1817.],
         [2144.],
         ...,
         [2828.],
         [2805.],
         [2668.]],

        [[1717.],
         [1776.],
         [1503.],
         ...,
         [2781.],
         [2457.],
         [2974.]],

        ...,

        [[4891.],
         [4603.],
         [4425.],
         ...,
         [3526.],
         [3198.],
         [3683.]],

        [[4835.],
        

ValueError: Invalid integer data type 'f'.

In [ ]:
output[0]